In [1]:
import argparse, os, datetime, pickle
from scipy.optimize import curve_fit
from scipy.special import erfc
import numpy as np
global aadict

three = ["ALA", "ARG", "ASN", "ASP", "CYS", "GLN", "GLU", "GLY", "HIS",
         "ILE", "LEU", "LYS", "MET", "PHE", "PRO", "SER", "THR", "TRP", "TYR", "VAL"]
one = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I",
       "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]
aadict = {}

for i in range(len(three)):
    aadict[three[i]] = one[i]
    aadict[one[i]] = three[i]

def one2three2one(aa):
    return(aadict[aa.upper()])

def sequence(pdb):
    file = open(pdb, "r")
    seq = ''
    for line in file:
        lsplit = line.split()
        if lsplit[0] == 'ATOM':
            tag, iatom, atom, res, ires = lsplit[:5]
            if atom == "CA":
                seq += one2three2one(res)
    file.close()
    return seq

def distance(r1, r2):
    return (np.sum((r1 - r2)**2))**0.5

def chargematrix(filename, nres):
    chargedresidues = ["ASP", "GLU", "HIS", "LYS", "ARG", "TYR", "CYS"]
    chargedatoms = ["CG", "CD", "CD2", "NZ", "CZ", "OH", "SG"]
    chargedict = {}
    for i, res in enumerate(chargedresidues):
        chargedict[res] = chargedatoms[i]

    file = open(filename, 'r')
    tmp = []
    residues = []
    for line in file:
        lsplit = line.split()
        if lsplit[0] == 'ATOM':
            tag, iatom, atom, res, ires = lsplit[:5]
            if int(ires) == 1:
                if atom == "N":
                    # print res, atom
                    (x, y, z) = (line[30:38], line[38:46], line[46:54])
                    tmp.append([float(x), float(y), float(z)])
                    residues.append("n0")
            if res in chargedict.keys():
                if atom == chargedict[res]:
                    # print res, atom
                    (x, y, z) = (line[30:38], line[38:46], line[46:54])
                    tmp.append([float(x), float(y), float(z)])
                    residues.append(one2three2one(res) + ires)
            if int(ires) == nres:
                if atom == "C":
                    # print res, atom
                    (x, y, z) = (line[30:38], line[38:46], line[46:54])
                    tmp.append([float(x), float(y), float(z)])
                    residues.append("c" + str(int(ires) + 1))
    file.close()
    chargecoors = np.array(tmp)

    return((np.array([[distance(i, j) for i in chargecoors] for j in chargecoors]), residues))


def CAmatrix(filename, ifirst=1, ilast=1000000):
    file = open(filename, 'r')
    tmp = []
    residues = []
    for line in file:
        lsplit = line.split()
        if lsplit[0] == 'ATOM':
            tag, iatom, atom, res, ires = lsplit[:5]
            if atom == "CA" and ifirst <= int(ires) <= ilast:
                # print res, atom
                (x, y, z) = (line[30:38], line[38:46], line[46:54])
                tmp.append([float(x), float(y), float(z)])
                residues.append(res + ires)
    file.close()
    CAcoors = np.array(tmp)

    return((np.array([[distance(i, j) for i in CAcoors] for j in CAcoors]), residues))


def get_diagonal(matrix, n=0):
    z = np.zeros(matrix.shape, int)
    for i in range(len(z) - n):
        z[i, i + n] = True
    return matrix[z > 0]


def smallmatrixlimits(ires, cutoff, len):
    ileft = max(1, ires - cutoff)
    iright = min(ileft + 2 * cutoff, len)
    if iright == len:
        ileft = max(1, iright - 2 * cutoff)
    return (ileft, iright)

def smallmatrixpos(ires, cutoff, len):
    resi = cutoff + 1
    if ires < cutoff + 1:
        resi = ires
    if ires > len - cutoff:
        resi = min(len, 2 * cutoff + 1) - (len - ires)
    return resi

def hasselbalch(pH, pK, nH):
    return (10 ** (pK - pH)) ** nH / (1. + (10 ** (pK - pH)) ** nH)

def k(ion, T, eps):
    return np.sqrt((8.0 * np.pi * ion * 0.2003457539870666) / (eps * T * 0.0019872041))

def W1(r, ion, T, eps):
    kappa = k(ion, T, eps)
    x = kappa * r / np.sqrt(6.0)
    return 332.286 * np.sqrt(6.0 / np.pi) * (1 - np.sqrt(np.pi) * x * np.exp(x ** 2) * erfc(x)) / (eps * r)

def WP(ze, c, ion, T, eps):
    """An energy correction term, added to account for the effect of protein concentration c on the overall free interaction energy in protein solution."""
    kappa = k(ion, T, eps)
    r = 11.8 / pow(c, 1.0 / 3.0)  # in Angstroms
    return ze * ze * np.exp(-kappa * r) / (eps * r)

def W(ds, ion, eps, weight=None):
    k = np.sqrt(ion) / 3.08
    ds[ds == 0] = 1
    tmp = 332.286 / (eps * ds) * np.exp(- k * ds)
    return np.average(tmp, axis=2, weights=weight)

def w2logp(x, R, T):
    return x * 4181.2 / (R * T * np.log(10))

# Introduced for benchmarking purposes

def genSeq(N):
    seq = ''
    for i in range(N):
        seq = seq + 'D'
    return seq

now = datetime.datetime.now()

parser = argparse.ArgumentParser()
parser.add_argument("--pdbdir", default='.')
parser.add_argument("--sequence", default='nMDVFMKGLSKAKEGVVAAAEKTKQGVAEAAGKTKEGVLYVGSKTKEGVVHGVATVAEKTKEQVTNVGGAVVTGVTAVAQKTVEGAGSIAAATGFVKKDQLGKNEEGAPQEGILEDMPVDPDNEAYEMPSEEGYQDYEPEAc')
parser.add_argument("--temperature", default=283.15)
parser.add_argument("--ionicstrength", default=0.0)
parser.add_argument("--epsilon", default=83.83)
parser.add_argument("--gca", default=5)
parser.add_argument("--gcb", default=7.5)
parser.add_argument("--cutoff", default=2)
parser.add_argument("--ncycles", default=3)
args = parser.parse_args()

print 'pepKalc paramteres:'
print (args)

pdbdir = args.pdbdir
# os.chdir(pdbdir)
# if pdbdir and pdbdir[-1] != '/': pdbdir += '/'
seq = args.sequence

temp = float(args.temperature)
ion = float(args.ionicstrength)
eps = float(args.epsilon)

gca = float(args.gca)
gcb = float(args.gcb)

cutoff = int(args.cutoff)
ncycles = int(args.ncycles)

# Init the array of epsilon values
EPSILON = [eps]

for E in EPSILON:
    # npdb=0
    eps = E
    pdbfiles = [f for f in os.listdir(pdbdir) if f[-3:] == 'pdb']
    if pdbfiles:
        npdb = len(pdbfiles)
        print(npdb, ' pdbfiles found in ', pdbdir)

        try:
            wfile = open(pdbdir + 'as_reduced_weights.dat', 'r')
            wdict = {}
            for line in wfile:
                lsplit = line.split()
                wdict['as_' + lsplit[0] + '.pdb'] = float(lsplit[1])
            wfile.close()
            print('weights from ', pdbdir + 'as_reduced_weights.dat')
            allweightsareone = False
        except:
            print('all weights are one')
            allweightsareone = True

        seq = 'n' + sequence(pdbdir + pdbfiles[0]) + 'c'
        print('sequence: ', seq)

    pK0 = {"n": 7.5, "C": 8.6, "D": 4.0, "E": 4.4,
           "H": 6.6, "K": 10.4, "R": 12.0, "Y": 9.6, "c": 3.5}

    pos = np.array([i for i in xrange(len(seq)) if seq[i] in pK0.keys()])
    sites = ''.join([seq[i] for i in pos])

    if pdbfiles:
        weight = np.ones(len(pdbfiles))
        dranges = []
        for ip, pdb in enumerate(pdbfiles):
            if not allweightsareone:
                weight[ip] = wdict[pdb]
            dranges.append(chargematrix(pdbdir + pdb, len(pos))[0])
        ds = np.array(dranges).transpose()
        tmp = W(ds, ion, eps, weight)
        # print(ds.shape)
    else:
        l = np.array([abs(pos - pos[i]) for i in range(len(pos))])
        d = gca + np.sqrt(l) * gcb
        tmp = W1(d, ion, temp, E)

    R = 8.314472
    pHstep = 0.2
    pHs = np.arange(1.0, 13.01, pHstep)

    neg = np.array([i for i in range(len(pos)) if seq[pos[i]] in 'CDEYc'])

    #print('distancematrix: ', ds.shape)
    I = np.diag(np.ones(len(pos)))
    tmp[I == 1] = 0
    ww = w2logp(tmp, R, temp) / 2

    chargesempty = np.zeros(pos.shape[0])
    if len(neg):
        chargesempty[neg] = -1

    pK0s = [pK0[c] for c in sites]
    nH0s = [0.9 for c in sites]

    titration = np.zeros((len(pos), len(pHs)))
    smallN = min(2 * cutoff + 1, len(pos))
    smallI = np.diag(np.ones(smallN))

    alltuples = [[int(c) for c in np.binary_repr(i, smallN)]
                 for i in xrange(2 ** (smallN))]
    gmatrix = [np.zeros((smallN, smallN)) for p in range(len(pHs))]

    if len(sites) <= 2 * cutoff + 1:
        ncycles = 1

    titration = np.array([[hasselbalch(pHs[p], pK0s[i], nH0s[i])
                           for i in range(len(pos))] for p in range(len(pHs))]).transpose()
    for icycle in range(ncycles):
        # print icycle + 1
        fractionhold = titration.transpose()

        for ires in range(1, len(pos) + 1):
            (ileft, iright) = smallmatrixlimits(ires, cutoff, len(pos))
            resi = smallmatrixpos(ires, cutoff, len(pos))

            for p in range(len(pHs)):
                fraction = fractionhold[p].copy()
                fraction[ileft - 1: iright] = 0
                charges = chargesempty + fraction
                ww0 = np.diag(np.dot(ww, charges) * 2)
                gmatrixfull = ww + ww0 + pHs[p] * I - np.diag(pK0s)
                gmatrix[p] = gmatrixfull[ileft - 1: iright, ileft - 1: iright]

            E_all = np.array([sum([10 ** -(gmatrix[p] * np.outer(c, c)).sum()
                                   for c in alltuples]) for p in range(len(pHs))])
            E_sel = np.array([sum([10 ** -(gmatrix[p] * np.outer(c, c)).sum()
                                   for c in alltuples if c[resi - 1] == 1]) for p in range(len(pHs))])
            titration[ires - 1] = E_sel / E_all

        sol = np.array([curve_fit(hasselbalch, pHs, titration[i], [
                       pK0s[i], nH0s[i]])[0] for i in range(len(pK0s))])
        (pKs, nHs) = sol.transpose()

    first = 0
    if not seq[0] == 'n':
        first = 1

    # Print the header
    print '%s, %s' % ('RES','dpKa')
    for i in range(len(sol)):
        print '%s, %f' % (seq[pos[i]] + str(pos[i] + first), pKs[i] - pK0s[i]) #pKs[i]#, pKs[i] - pK0s[i], nHs[i]
        outfile = open(seq[pos[i]] + str(pos[i] + first) + '_' + str(E) + '.dat', 'w')
        for p in range(len(pHs)):
            outfile.write('%7.7f, %7.7f\n' % (pHs[p], titration[i][p]))
        outfile.close()

# pickle.dump((pK0, pHs, pK0s, pKs, nHs, titration, seq, pos, temp, ion, eps, gca, gcb, pdbdir, npdb),open('plotdata','wb'))
#
# outfile = open('outfile', 'w')
# outfile.write('date: ' + now.strftime("%Y-%m-%d %H:%M") + '\n\n')
# outfile.write(str((pdbdir, seq, temp, ion, eps, gca, gcb, cutoff, ncycles)) + '\n\n')
# if pdbfiles:
#     if not allweightsareone:
#         outfile.write('weights from ' + pdbdir + 'as_reduced_weights.dat\n\n')
#     else:
#         outfile.write('all weights are the same\n\n')
#
# lines=["\n"]
# for i in range(len(pos)):
#     lines.append( "%s: %7.3f %7.3f %7.3f \n" %(seq[pos[i]]+str(pos[i] + first), pKs[i], pKs[i] - pK0s[i], nHs[i]) )
# outfile.writelines(lines)
#
# outfile.close()

usage: ipykernel_launcher.py [-h] [--pdbdir PDBDIR] [--sequence SEQUENCE]
                             [--temperature TEMPERATURE]
                             [--ionicstrength IONICSTRENGTH]
                             [--epsilon EPSILON] [--gca GCA] [--gcb GCB]
                             [--cutoff CUTOFF] [--ncycles NCYCLES]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/kamil/Library/Jupyter/runtime/kernel-f713c15c-7fa1-4496-ac26-4e4cc7adb0be.json


SystemExit: 2

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
